In [18]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt 
import textwrap
from matplotlib.dates import DateFormatter

In [19]:
def customer_identifier(df): 
    '''
        It handles strings in the "Customer" column of the Products DataFrame.
        "Customer" values are strings with '-' character splitting customer names from countries and/or 
        another specifications such as competitor product/brand/company. 
        This method splits the string, maintaining at the Customer column only the actual customer name.
        The remaining info is reallocated to the new columns "Specifics" and "Competitor".
        It only applies to product DataFrame in this specific context!

        Arguments:
            df: DataFrame (product DataFrame)

        Returns:
            df_result: DataFrame (resulting DataFrame after string handling)
    '''

    df_result = df
    df_result["Competitor"] = 0
    df_result["Specifics"] = df_result["Customer"]
    df_result["Specifics"] = df_result["Specifics"].str.replace(r'^.+ - ','')
    df_result["Customer"] = df_result["Customer"].str.replace(r' - .+$','')

    # All character uppercased 
    df_result.loc[(df_result["Customer"].str.fullmatch(r'[A-Z ]{3,}')),"Customer"] = \
        df_result.loc[(df_result["Customer"].str.fullmatch(r'[A-Z ]{3,}')),"Customer"].str.title()

    df_result.loc[(df_result["Specifics"].str.find("Concorrente")!=-1),"Competitor"] = 1
    df_result.loc[(df_result["Specifics"].str.find("Concorrente")!=-1),"Specifics"].str.replace("Concorrente",'')
    
    return df_result


def verify_inconsistences(df):
    '''
        It verifies some inconsistences in the DataFrame, such as the following situations:

        1. Uncompatible 'Availabe'-'Unavailable' values
        2. 'RandomPrecosNegativos' indicating positive final price while 
            'FinalPrice' value is actually negative
        3. 'RandomPrecosMissing' indicating missing final price while 'FinalPrice'
            value is not missing, or 'RandomPrecosMissing' indicating no missing value
            while 'FinalPrice' value is actually missing

        Arguments:
            df: DataFrame ('coletas' DataFrame)

        Returns:
            None
    '''

    # Situation 1
    assert df[(df["Available"]==df["Unavailable"])].empty,\
    "Inconsistencies with 'Available 'Unavailable' values!"
    
    # Situation 2
    assert df[(df["RandomPrecosNegativos"]==1)&(df["FinalPrice"]<0)].empty,\
    "Inconsistencies with negative final price values!"

    # Situation 3
    assert df.loc[
    ((df["RandomPrecosMissing"] == 0)&(df["FinalPrice"] != 0))|
    ((df["RandomPrecosMissing"] == 1)&(df["FinalPrice"] == 0)),"FinalPrice"
    ].empty, "Inconsistencies with values ​​missing from final prices!"
    
    return None


def fix_inconsistences(df, key_error):
    '''
        It fixes some inconsistences in the DataFrame, indicated by the 'key_error' string.

        1. 'missing':  If the Final Price is missing, we get Suggested Price instead
        2. 'negative': If the Final Price is negative, we get the mean of same EAN 
            positive Final Prices instead

        Arguments:
            df_coleta: DataFrame ('coletas' DataFrame)
            key_error: str (string indicating which type of inconsistence we're dealing with)

        Returns:
            df_coleta: DataFrame ('coletas' DataFrame with modifications)
    '''

    df_result = df
    # Situation 1
    if (key_error == 'missing'):
        df_result.loc[((df_result["RandomPrecosMissing"] == 0)|(df_result["FinalPrice"] == 0)),"FinalPrice" ] = \
        df_result.loc[((df_result["RandomPrecosMissing"] == 0)|(df_result["FinalPrice"]==0)),"SuggestedPrice"]
        df_result.loc[((df_result["RandomPrecosMissing"] == 0)|(df_result["FinalPrice"] == 0)),"RandomPrecosMissing"] = 1

    # Situation 2
    elif(key_error == 'negative'):
        '''mean_prices = pd.DataFrame(df_result.loc[(df["FinalPrice"] > 0),
        ["FinalPrice","MasterKey_RetailerProduct"]].groupby("MasterKey_RetailerProduct").mean())


        for item in df_result.loc[(df_result["FinalPrice"] < 0),["FinalPrice","MasterKey_RetailerProduct"]].index.values:
            df_result.loc[item,"FinalPrice"] = mean_prices.loc[str(df_result.loc[item,"MasterKey_RetailerProduct"])]["FinalPrice"]
            df_result.loc[item,"RandomPrecosNegativos"] = 1
        '''    
        df_result.loc[(df_result["FinalPrice"] < 0),"FinalPrice"] = df_result.loc[(df_result["FinalPrice"] < 0),"FinalPrice"].abs()

    return df_result

In [20]:
# Folder path
path =  "../data/"

# Which values are considered NaN
na_vls = ['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', 
               '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 
                'null', '...']

nrows = 200000

# Load datasets
'''df_prod = pd.read_csv(path+"ProdutosVarejos.csv", na_values = na_vls, nrows = nrows)
df_coletas = pd.read_csv(path+"Coletas.csv", na_values = na_vls, nrows = nrows)'''

df_prod = pd.read_csv(path+"ProdutosVarejos.csv", na_values = na_vls)
df_coletas = pd.read_csv(path+"Coletas.csv", na_values = na_vls)

# Handling missing values
df_coletas.dropna(inplace=True, how = 'all')
df_prod.dropna(inplace=True, how = 'all')
df_prod.fillna(value = '', inplace=True)
df_coletas.fillna(value = 0, inplace=True)

# Register DataFrames Shapes
clt_cols, clt_rows = df_coletas.shape
prd_cols, prod_rows = df_prod.shape

# Types for each DataFrame column
prod_types = {'Customer': 'str',
                'Department': 'str',
                'Category': 'str',
                'Brand': 'str',
                'EAN': 'str',
                'Product': 'str',
                'Retailer': 'str',
                'MasterKey_RetailerProduct': 'str'}

coletas_types = {'DateIns': 'str', 
                    'Screenshot': 'str', 
                    'Available': 'int64', 
                    'Unavailable': 'int64', 
                    'SuggestedPrice' : 'float64',
                    'FinalPrice': 'float64', 
                    'MasterKey_RetailerProduct': 'str', 
                    'RandomPrecosNegativos': 'int64',
                    'RandomPrecosMissing': 'int64', 
                    'RandomPrecosDiscrepantes': 'int64',
                    'RandomPrecosDiscrepantesFator': 'float64'}

# Casting DataFrames columns according to the types above
df_prod = df_prod.astype(prod_types)
df_coletas= df_coletas.astype(coletas_types)

# Adjusting strings in "Customer" column
df_prod = customer_identifier(df_prod)

# Transforming 'DateIns' column in a Pandas datetype time series
df_coletas["DateIns"] = pd.to_datetime(df_coletas["DateIns"].values)

# Formatting "Department","Category", "Product" columns strings to begin (each word) with upper case 

for cols in df_prod[["Department","Category"]].columns.values:
    df_prod[cols] = df_prod[cols].str.capitalize()

# Formatting "Brand" column value strings to begin with upper case  
# All character uppercased 
df_prod.loc[df_prod["Brand"].str.fullmatch(r'[A-Z ]{3,}'),"Brand"] = \
    df_prod.loc[df_prod["Brand"].str.fullmatch(r'[A-Z ]{3,}'),"Brand"].str.title()


C:\Users\luiza\AppData\Local\Temp\ipykernel_2908\2574764220.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df_result["Specifics"] = df_result["Specifics"].str.replace(r'^.+ - ','')
C:\Users\luiza\AppData\Local\Temp\ipykernel_2908\2574764220.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_result["Customer"] = df_result["Customer"].str.replace(r' - .+$','')


In [21]:
# Inconsistences fixing and verification
df_coletas = fix_inconsistences(df_coletas, 'missing')
df_coletas = fix_inconsistences(df_coletas, 'negative')

In [22]:
# Saving Clean Data
df_prod = df_prod[['Customer', 'Department', 'Category',
                    'Brand', 'EAN', 'Product', 'Retailer',
                    'Specifics','Competitor',
                    'MasterKey_RetailerProduct']]
df_coletas = df_coletas[['DateIns', 'Screenshot', 
                        'Available', 'Unavailable', 
                        'SuggestedPrice','FinalPrice', 
                        'MasterKey_RetailerProduct']]
                        
df_coletas.to_csv(path_or_buf=path+"New_Coletas.csv",  columns=df_coletas.columns.values)
df_prod.to_csv(path_or_buf=path+"New_ProdutosVarejos.csv",  columns=df_prod.columns.values)